In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print(os.listdir("../input/facial-expression/fer2013/"))

# Any results you write to the current directory are saved as output.

In [ ]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE = 48
CLIPED_SIZE = 42
EMO_NUM = 7
BATCH_SIZE = 50
NUM_CHANNEL = 1
EPOCHS = 50

In [ ]:
# get the data
filname = '../input/facial-expression/fer2013/fer2013.csv'
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
names = ['emotion', 'pixels', 'usage']
df = pd.read_csv('../input/facial-expression/fer2013/fer2013.csv', names=names, na_filter=False)
im = df['pixels']
df.head(10)

In [ ]:
def getData(filname):
    # images are 48x48
    # N = 35887
    Y = []
    X = []
    first = True
    for line in open(filname):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])

    X, Y = np.array(X) / 255.0, np.array(Y)
    return X, Y

In [ ]:
X, Y = getData(filname)
num_class = len(set(Y))
print(num_class)

In [ ]:
# keras with tensorflow backend
N, D = X.shape
X = X.reshape(N, 48, 48, 1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
y_train = (np.arange(num_class) == y_train[:, None]).astype(np.float32)
y_test = (np.arange(num_class) == y_test[:, None]).astype(np.float32)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
# 进行左右对称
def GetSymmetric(pixel, size):
    '''
    pixel: np array with shape (count,size,size,1)
    '''
    count = pixel.shape[0]
    sym = np.zeros((count, size, size, NUM_CHANNEL))
    for i in range(count):
        for j in range(size):
            for k in range(size):
                sym[i,j,k,0] = pixel[i,j,size-k-1,0]
    return sym

In [ ]:
symmetric_x_train = GetSymmetric(X_train, IMAGE_SIZE)
X_train = np.concatenate((X_train, symmetric_x_train), axis = 0)
y_train = np.concatenate((y_train, y_train))
print(X_train.shape)
print(y_train.shape)

In [ ]:
import random

# Cliped
def GetClipedImage(pixel, start):
    '''
    pixel: raw 48*48 pixel data with shape (count, 48, 48, 1)
    start: a tuple such as (0,0),(2,3),(4,2), represents start point of clipped 42*42 image
    '''
    count = pixel.shape[0]
    out = np.zeros((count, CLIPED_SIZE, CLIPED_SIZE, NUM_CHANNEL))
    for i in range(count):
        for j in range(CLIPED_SIZE):
            out[i,j,:,0] = pixel[i,start[0]+j,start[1]:start[1]+CLIPED_SIZE,0]
    return out
# To process
def DataPreprocess(pixel, label = []):
    '''
    pixel: pixel data with shape (count,48,48,1)
    label: optical, corresponding label of pixel
    '''
    a = random.randint(0,2)
    b = random.randint(3,5)
    c = random.randint(0,2)
    d = random.randint(3,5)
    pixel1 = GetClipedImage(pixel, (a,c))
    pixel2 = GetClipedImage(pixel, (a,d))
    pixel3 = GetClipedImage(pixel, (b,c))
    pixel4 = GetClipedImage(pixel, (b,d))
    out_p = np.concatenate((pixel1, pixel2, pixel3, pixel4), axis = 0)
    if len(label) == 0:
        return out_p
    else:
        out_l = np.concatenate((label, label, label, label), axis = 0)
        return (out_p, out_l)

In [ ]:
(X_train, y_train) = DataPreprocess(X_train, y_train)
(X_test, y_test) = DataPreprocess(X_test, y_test)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
# 数据sao操作
datagen = ImageDataGenerator(
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization

In [ ]:
def my_model():
    model = Sequential()
    
    input_shape = (42, 42, 1)
    
    model=Sequential()
    model.add(Conv2D(32,(1,1),strides=1,padding='same',input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(5,5),padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(32,(3,3),padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
 
    model.add(Conv2D(64,(5,5),padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
        
    model.add(Flatten())
    model.add(Dense(2048))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7))
    model.add(Activation('softmax'))
    model.summary()

    return model

In [ ]:
from keras import backend as K
from tensorflow.keras.applications.resnet50 import ResNet50

path_model='model_filter.h5' # save model at this location after each epoch
K.tensorflow_backend.clear_session() # destroys the current graph and builds a new one
# model=my_model() # create the model

model = my_model()

# K.set_value(model.optimizer.lr,1e-3) # set the learning rate

model.compile(optimizer=adam(lr=1e-3), loss="mae")

# fit the model
h=model.fit_generator(datagen.flow(X_train, y_train, batch_size=128),
                    epochs=50,
                    verbose=1,
                    validation_data=(X_test,y_test),
                    callbacks=[
                        ModelCheckpoint(filepath=path_model),
                    ]
                   )

# h=model.fit(x=X_train,     
#             y=y_train, 
#             batch_size=64, 
#             epochs=20, 
#             verbose=1, 
#             validation_data=(X_test,y_test),
#             shuffle=True,
#             callbacks=[
#                 ModelCheckpoint(filepath=path_model),
#             ]
#             )

In [ ]:
objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
y_pos = np.arange(len(objects))
print(y_pos)

In [ ]:
# def emotion_analysis(emotions):
#     objects = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
#     y_pos = np.arange(len(objects))
#     plt.bar(y_pos, emotions, align='center', alpha=0.9)
#     plt.tick_params(axis='x', which='both', pad=10,width=4,length=10)
#     plt.xticks(y_pos, objects)
#     plt.ylabel('percentage')
#     plt.title('emotion')
    
# plt.show()

def emotion_analysis(emotions):
    objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    y_pos = np.arange(len(objects))
 
    plt.bar(y_pos, emotions, align='center', alpha=0.5)
    plt.xticks(y_pos, objects)
    plt.ylabel('percentage')
    plt.title('emotion')
 
    plt.show()

In [ ]:
y_pred=model.predict(X_test)
# print(y_pred)
y_test.shape

In [ ]:
#import seaborn as sn
#import pandas as pd
#import matplotlib.pyplot as plt
#import numpy as np
#from sklearn.metrics import confusion_matrix
#%matplotlib inline
#cm = confusion_matrix(np.where(y_test == 1)[1], y_pred)
#cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#df_cm = pd.DataFrame(cm, index = [i for i in "0123456"],
                  #columns = [i for i in "0123456"])
#plt.figure(figsize = (20,15))
#sn.heatmap(df_cm, annot=True)

In [ ]:
from skimage import io
img = image.load_img('../input/testimages/myself.jpg', grayscale=True, target_size=(42, 42))
show_img=image.load_img('../input/testimages/myself.jpg', grayscale=False, target_size=(200, 200))
x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)

x /= 255

custom = model.predict(x)
print(custom[0])
emotion_analysis(custom[0])

x = np.array(x, 'float32')
x = x.reshape([42, 42])

plt.gray()
plt.imshow(show_img)
plt.show()

m=0.000000000000000000001
a=custom[0]
for i in range(0,len(a)):
    if a[i]>m:
        m=a[i]
        ind=i
        
print('Expression Prediction:',objects[ind])